# X-Ray CNN Neural Network

### Authors: Blaise Swartwood, Brian Beasley, Tommaso Calviello

In [1]:
import torch
import torch.nn as nn
import torch.optim as optim
from torchvision import datasets, models, transforms
from torch.utils.data import DataLoader
import os
from sklearn.metrics import confusion_matrix, ConfusionMatrixDisplay
import matplotlib.pyplot as plt

In [2]:
data_transforms = {
    'train': transforms.Compose([
        transforms.Resize((224, 224)),  # ResNet expects 224x224 images
        transforms.RandomHorizontalFlip(),
        transforms.ToTensor(),
        transforms.Normalize([0.485, 0.456, 0.406], [0.229, 0.224, 0.225])  # Normalization for pre-trained models
    ]),
    'val': transforms.Compose([
        transforms.Resize((224, 224)),
        transforms.ToTensor(),
        transforms.Normalize([0.485, 0.456, 0.406], [0.229, 0.224, 0.225])
    ]),
    'test': transforms.Compose([
        transforms.Resize((224, 224)),
        transforms.ToTensor(),
        transforms.Normalize([0.485, 0.456, 0.406], [0.229, 0.224, 0.225])
    ]),
}

In [4]:
data_dir = '../data/chest_xray'
image_datasets = {
    'train': datasets.ImageFolder(os.path.join(data_dir, 'train'), data_transforms['train']),
    'val': datasets.ImageFolder(os.path.join(data_dir, 'val'), data_transforms['val']),
    'test': datasets.ImageFolder(os.path.join(data_dir, 'test'), data_transforms['test'])
}

dataloaders = {
    'train': DataLoader(image_datasets['train'], batch_size=32, shuffle=True),
    'val': DataLoader(image_datasets['val'], batch_size=32, shuffle=False),
    'test': DataLoader(image_datasets['test'], batch_size=32, shuffle=False)
}

dataset_sizes = {x: len(image_datasets[x]) for x in ['train', 'val', 'test']}
class_names = image_datasets['train'].classes  # This will be ['normal', 'pneumonia_bacteria', 'pneumonia_virus']

print(f"Class Names: {class_names}")
print(f"Dataset Sizes: {dataset_sizes}")

FileNotFoundError: Found no valid file for the classes PNEUMONIA. Supported extensions are: .jpg, .jpeg, .png, .ppm, .bmp, .pgm, .tif, .tiff, .webp

In [ ]:
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
print(device)

model = models.resnet50(pretrained=True)
num_ftrs = model.fc.in_features
model.fc = nn.Sequential(
    nn.Dropout(p=0.5),  # Dropout layer for regularization
    nn.Linear(num_ftrs, 2)  # Binary classification layer
)
model = model.to(device)

# Criterion and optimizer
criterion = nn.CrossEntropyLoss()
optimizer = optim.SGD(model.parameters(), lr=0.001, momentum=0.9, weight_decay=1e-4)  # Weight decay added
scheduler = optim.lr_scheduler.ReduceLROnPlateau(optimizer, mode='min', factor=0.1, patience=3, verbose=True) #Reducing the learning rate over time

In [ ]:
# Training function with learning rate scheduling
def train_model(model, criterion, optimizer, scheduler, num_epochs=2):
    for epoch in range(num_epochs):
        print(f'Epoch {epoch+1}/{num_epochs}')
        print('-' * 10)

        for phase in ['train', 'val']:
            if phase == 'train':
                model.train()  # Training mode
            else:
                model.eval()  # Evaluation mode

            running_loss = 0.0
            running_corrects = 0

            # Iterate over data
            for inputs, labels in dataloaders[phase]:
                inputs = inputs.to(device)
                labels = labels.to(device)

                # Zero the parameter gradients
                optimizer.zero_grad()

                # Forward pass
                with torch.set_grad_enabled(phase == 'train'):
                    outputs = model(inputs)
                    _, preds = torch.max(outputs, 1)
                    loss = criterion(outputs, labels)

                    # Backward pass and optimize in training mode
                    if phase == 'train':
                        loss.backward()
                        optimizer.step()

                # Statistics
                running_loss += loss.item() * inputs.size(0)
                running_corrects += torch.sum(preds == labels.data)

            epoch_loss = running_loss / dataset_sizes[phase]
            epoch_acc = running_corrects.double() / dataset_sizes[phase]

            print(f'{phase} Loss: {epoch_loss:.4f} Acc: {epoch_acc:.4f}')

            # Adjust learning rate based on validation loss
            if phase == 'val':
                scheduler.step(epoch_loss)

    print('Training complete')

# Testing function
def test_model(model):
    model.eval()
    running_corrects = 0
    all_preds = []
    all_labels = []

    with torch.no_grad():
        for inputs, labels in dataloaders['test']:
            inputs = inputs.to(device)
            labels = labels.to(device)

            outputs = model(inputs)
            _, preds = torch.max(outputs, 1)

            running_corrects += torch.sum(preds == labels.data)
            all_preds.extend(preds.cpu().numpy())
            all_labels.extend(labels.cpu().numpy())

    test_acc = running_corrects.double() / dataset_sizes['test']
    print(f'Test Accuracy: {test_acc:.4f}')

    # Confusion matrix
    cm = confusion_matrix(all_labels, all_preds)
    disp = ConfusionMatrixDisplay(confusion_matrix=cm, display_labels=class_names)
    disp.plot(cmap=plt.cm.Blues)
    plt.title("Pneumonia X-Ray Confusion Matrix")
    plt.show()

    calculate_metrics(cm)

def calculate_metrics(cm):
    # Calculate metrics
    precision = []
    recall = []
    f1_score = []
    
    for i in range(len(cm)):
        TP = cm[i, i]
        FP = sum(cm[j, i] for j in range(len(cm))) - TP
        FN = sum(cm[i, j] for j in range(len(cm))) - TP
        
        # Calculate precision, recall, and F1 score for each class
        precision_i = TP / (TP + FP) if (TP + FP) > 0 else 0
        recall_i = TP / (TP + FN) if (TP + FN) > 0 else 0
        f1_score_i = 2 * precision_i * recall_i / (precision_i + recall_i) if (precision_i + recall_i) > 0 else 0
        
        precision.append(precision_i)
        recall.append(recall_i)
        f1_score.append(f1_score_i)
    
    # Display per-class precision, recall, and F1 scores
    for i, class_name in enumerate(class_names):
        print(f"Class {class_name} - Precision: {precision[i]*100:.2f}%, "
              f"Recall: {recall[i]*100:.2f}%, "
              f"F1 Score: {f1_score[i]*100:.2f}%")

# Save model function
def save_model(model, path='resnet50_pneumonia.pth'):
    torch.save(model.state_dict(), path)
    print(f"Model saved as {path}")

In [ ]:
train_model(model, criterion, optimizer, scheduler)

In [ ]:
test_model(model)

In [ ]:
save_model(model)

Precision
The percentage of correct predictions out of all predictions for each class.

Recall (Sensitivity)
The percentage of actual instances of a class that were correctly predicted.

F1-score
The harmonic mean of precision and recall.